In [28]:
import xml.etree.ElementTree as xet
from glob import glob
import os
import numpy as np
from ultralytics import YOLO
import torch
from PIL import Image, ImageOps
import easyocr
import cv2

In [11]:
path = glob('./non_ev_vehicle_dataset/annotations/*xml')
labels_dir = './data/labels/'
class_id = 1
os.makedirs(labels_dir, exist_ok=True)

In [12]:
for filename in path:
    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)
    
    img_width = int(root.find('size').find('width').text)
    img_height = int(root.find('size').find('height').text)
    
    center_x = (xmin + xmax) / 2.0 / img_width
    center_y = (ymin + ymax) / 2.0 / img_height
    bbox_width = (xmax - xmin) / img_width
    bbox_height = (ymax - ymin) / img_height
    
    data = f"{class_id} {center_x} {center_y} {bbox_width} {bbox_height}\n"
    
    base_filename = os.path.basename(os.path.splitext(filename)[0])
    txt_filename = os.path.join(labels_dir, base_filename + '.txt')
    with open(txt_filename, 'w') as f:
        f.write(data)